In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
os.chdir('/content/gdrive/MyDrive/')

Mounted at /content/gdrive


In [ ]:
%cd 'Hate_Speech/'

/content/gdrive/MyDrive/Hate_Speech


In [ ]:
# %cd ..

In [ ]:
!ls

'='		 data_utils.py			 model.py
 classifier.pt	 hate_speech_spanish_scl.ipynb	 __pycache__
 config.py	 loss_func.py			 README.md
 dataset	 main.py			 requirements.txt


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: 

In [ ]:
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.4 MB/s eta 0:00:00
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from model import Transformer
from loss_func import CELoss, SupConLoss, DualLoss
from data_utils import load_data
from transformers import TFMT5Model, T5Tokenizer, logging, AutoTokenizer, AutoModel, AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import os
import sys
import time
import random
from tqdm import tqdm
import logging
import argparse
from datetime import datetime
import numpy as np
import json
from functools import partial
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
class CELoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.xent_loss(outputs['predicts'], targets)

class ConLoss(nn.Module):

    def __init__(self, alpha, temp):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            # delete diag elem
            mask = mask ^ torch.diag_embed(torch.diag(mask))
        # compute logits
        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp
        # delete diag elem
        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()
        # compute log prob
        exp_logits = torch.exp(logits)
        # mask out positives
        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        # in case that mask.sum(1) is zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)
        # compute log-likelihood
        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        # ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss = self.alpha * self.nt_xent_loss(normed_cls_feats, normed_cls_feats, targets)
        return cl_loss

class SupConLoss(nn.Module):

    def __init__(self, alpha, temp):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            # delete diag elem
            mask = mask ^ torch.diag_embed(torch.diag(mask))
        # compute logits
        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp
        # delete diag elem
        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()
        # compute log prob
        exp_logits = torch.exp(logits)
        # mask out positives
        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        # in case that mask.sum(1) is zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)
        # compute log-likelihood
        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss = self.alpha * self.nt_xent_loss(normed_cls_feats, normed_cls_feats, targets)
        return ce_loss + cl_loss

class AdversarialConLoss(nn.Module):
    def __init__(self, alpha, temp):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            # delete diag elem
            mask = mask ^ torch.diag_embed(torch.diag(mask))
        # compute logits
        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp
        # delete diag elem
        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()
        # compute log prob
        exp_logits = torch.exp(logits)
        # mask out positives
        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        # in case that mask.sum(1) is zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)
        # compute log-likelihood
        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, outputs, targets):

        ce_loss = self.xent_loss(outputs['predicts'], targets)
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)

        unnormalized_noise = normed_cls_feats.clone()
        norm = unnormalized_noise.norm(p=2, dim=-1)
        normalized_noise = unnormalized_noise / (norm.unsqueeze(dim=-1) + 1e-10)  # add 1e-10 to avoid Nan

        noise_norm = 1.5 # or 0.00005
        alp = 0.5

        target_noise = noise_norm * normalized_noise
        noise_x_ = normed_cls_feats + target_noise
        noise_CEloss = self.xent_loss(noise_x_, targets)
        noise_scloss =self.nt_xent_loss(normed_cls_feats, noise_x_, targets)
        # noise_CEloss = F.nll_loss(F.log_softmax(self.fc(noise_x_), dim=1), data.y)

        #noise_loss = (1 - self.alpha) * noise_CEloss + self.alpha * noise_scloss

        scl_loss = self.alpha * self.nt_xent_loss(normed_cls_feats, normed_cls_feats, targets)
        total_loss = (1 - self.alpha) *(ce_loss + noise_CEloss)/2 + noise_scloss * self.alpha

        return  total_loss


class DualLoss(SupConLoss):

    def __init__(self, alpha, temp):
        super().__init__(alpha, temp)

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)

        normed_label_feats = F.normalize(outputs['label_feats'], dim=-1)
        normed_pos_label_feats = torch.gather(normed_label_feats, dim=1, index=targets.reshape(-1, 1, 1).expand(-1, 1, normed_label_feats.size(-1))).squeeze(1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(normed_pos_label_feats, normed_cls_feats, targets)
        cl_loss_2 = 0.5 * self.alpha * self.nt_xent_loss(normed_cls_feats, normed_pos_label_feats, targets)
        return ce_loss + cl_loss_1 + cl_loss_2

In [ ]:
class Instructor:

    def __init__(self, args, logger):
        self.args = args
        self.logger = logger
        self.logger.info('> creating model {}'.format(args.model_name))
        if args.model_name == 'xlm-roberta':
            # self.tokenizer = AutoTokenizer.from_pretrained('berta-base-multilingual-cased')
            self.tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
            # self.tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

            # tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
            # model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

            # base_model = AutoModel.from_pretrained('bert-base-multilingual-cased')
            base_model = AutoModel.from_pretrained("xlm-roberta-base")

            # base_model = TFMT5Model.from_pretrained('google/mt5-small')
            # self.tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

            # model = TFMT5Model.from_pretrained("google/mt5-small")
            # tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
        elif args.model_name == 'roberta':
            self.tokenizer = AutoTokenizer.from_pretrained('roberta-base-multilingual-cased', add_prefix_space=True)
            base_model = AutoModel.from_pretrained('roberta-base--multilingual-cased')
        elif args.model_name == 'mbert':
            self.tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', add_prefix_space=True)
            base_model = AutoModel.from_pretrained('bert-base-multilingual-cased')
        else:
            raise ValueError('unknown model')
        self.model = Transformer(base_model, args.num_classes, args.method)
        self.model.to(args.device)
        if args.device.type == 'cuda':
            self.logger.info('> cuda memory allocated: {}'.format(torch.cuda.memory_allocated(args.device.index)))
        self._print_args()

    def _print_args(self):
        self.logger.info('> training arguments:')
        for arg in vars(self.args):
            self.logger.info(f">>> {arg}: {getattr(self.args, arg)}")

    def _train(self, dataloader, criterion, optimizer):
        train_loss, n_correct, n_train = 0, 0, 0
        self.model.train()
        predictions, true_vals = [], []
        for inputs, targets in tqdm(dataloader, disable=self.args.backend, ascii=' >='):
            inputs = {k: v.to(self.args.device) for k, v in inputs.items()}
            targets = targets.to(self.args.device)
            outputs = self.model(**inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * targets.size(0)
            n_correct += (torch.argmax(outputs['predicts'], -1) == targets).sum().item()
            n_train += targets.size(0)

            predictions.append(outputs)
            true_vals.append(targets)

        return train_loss/n_train, n_correct/n_train, true_vals, predictions

    def _test(self, dataloader, criterion):
        test_loss, n_correct, n_test = 0, 0, 0
        self.model.eval()
        with torch.no_grad():
            predictions, true_vals = [], []
            for inputs, targets in tqdm(dataloader, disable=self.args.backend, ascii=' >='):
                inputs = {k: v.to(self.args.device) for k, v in inputs.items()}
                targets = targets.to(self.args.device)
                outputs = self.model(**inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item() * targets.size(0)
                n_correct += (torch.argmax(outputs['predicts'], -1) == targets).sum().item()
                n_test += targets.size(0)
                predictions.append(outputs)
                true_vals.append(targets)
        return test_loss/n_test, n_correct/n_test, true_vals, predictions

    def run(self):
        train_dataloader, test_dataloader = load_data(dataset=self.args.dataset,
                                                      data_dir=self.args.data_dir,
                                                      tokenizer=self.tokenizer,
                                                      train_batch_size=self.args.train_batch_size,
                                                      test_batch_size=self.args.test_batch_size,
                                                      model_name=self.args.model_name,
                                                      method=self.args.method,
                                                      workers=0)

        _params = filter(lambda p: p.requires_grad, self.model.parameters())
        if self.args.method == 'ce':
            criterion = CELoss()
        elif self.args.method == 'scl':
            criterion = SupConLoss(self.args.alpha, self.args.temp)
        elif self.args.method=='Adversarial':
             criterion = AdversarialConLoss(self.args.alpha, self.args.temp)
        elif self.args.method == 'cl':
            criterion =ConLoss(self.args.alpha, self.args.temp)
        elif self.args.method == 'dualcl':
            criterion = DualLoss(self.args.alpha, self.args.temp)
        else:
            raise ValueError('unknown method')
        optimizer = torch.optim.AdamW(_params, lr=self.args.lr, weight_decay=self.args.decay)
        best_loss, best_acc = 0, 0
        for epoch in range(self.args.num_epoch):
            train_loss, train_acc, targets, outputs = self._train(train_dataloader, criterion, optimizer)
            pred = []
            for o in outputs:
                pred.append(torch.argmax(o['predicts'], -1).detach().cpu().numpy())
            predictions = np.concatenate(pred, axis=0)
            tt = []
            for t in targets:
                tt.append(t.detach().cpu().numpy())
            true_vals = np.concatenate(tt, axis=0)
            print('Training phase', classification_report(predictions, true_vals, digits=4))
            test_loss, test_acc, targets, outputs = self._test(test_dataloader, criterion)
            pred = []
            for o in outputs:
                pred.append(torch.argmax(o['predicts'], -1).detach().cpu().numpy())
            predictions = np.concatenate(pred, axis=0)
            tt = []
            for t in targets:
                tt.append(t.detach().cpu().numpy())
            true_vals = np.concatenate(tt, axis=0)
            print('Test phase', classification_report(predictions, true_vals, digits=4))
            if test_acc > best_acc or (test_acc == best_acc and test_loss < best_loss):
                best_acc, best_loss = test_acc, test_loss
            self.logger.info('{}/{} - {:.2f}%'.format(epoch+1, self.args.num_epoch, 100*(epoch+1)/self.args.num_epoch))
            self.logger.info('[train] loss: {:.4f}, acc: {:.2f}'.format(train_loss, train_acc*100))
            self.logger.info('[test] loss: {:.4f}, acc: {:.2f}'.format(test_loss, test_acc*100))
        self.logger.info('best loss: {:.4f}, best acc: {:.2f}'.format(best_loss, best_acc*100))
        self.logger.info('log saved: {}'.format(self.args.log_name))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-f')
num_classes = {'hs': 2, 'turkish': 2, 'arabic': 2, 'pc': 2, 'cr': 2}
''' Base '''

parser.add_argument('--data_dir', type=str, default='data')
parser.add_argument('--dataset', type=str, default='arabic', choices=num_classes.keys())
parser.add_argument('--model_name', type=str, default='mbert', choices=['bert', 'roberta','mbert','xlm-roberta'])
parser.add_argument('--method', type=str, default='Adversarial', choices=['ce', 'scl', 'dualcl','Adversarial'])
''' Optimization '''
parser.add_argument('--train_batch_size', type=int, default=16)
parser.add_argument('--test_batch_size', type=int, default=16)
parser.add_argument('--num_epoch', type=int, default=20)
parser.add_argument('--lr', type=float, default=1e-5)
parser.add_argument('--decay', type=float, default=0.01)
parser.add_argument('--alpha', type=float, default=0.5)
parser.add_argument('--temp', type=float, default=0.1)
''' Environment '''
parser.add_argument('--backend', default=False, action='store_true')
parser.add_argument('--timestamp', type=int, default='{:.0f}{:03}'.format(time.time(), random.randint(0, 999)))
parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
args = parser.parse_args()
args.num_classes = num_classes[args.dataset]
args.device = torch.device(args.device)
args.log_name = '{}_{}_{}_{}.log'.format(args.dataset, args.model_name, args.method, datetime.now().strftime('%Y-%m-%d_%H-%M-%S')[2:])
if not os.path.exists('logs'):
    os.mkdir('logs')
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.addHandler(logging.FileHandler(os.path.join('logs', args.log_name)))

# **zero-shot**

In [ ]:
class MyDataset(Dataset):

    def __init__(self, raw_data, label_dict, tokenizer, model_name, method):
        label_list = ['positive', 'negative'] if method not in ['ce', 'scl'] else []
        sep_token = ['[SEP]'] if model_name == 'bert' else ['</s>']
        dataset = list()
        data=raw_data
        for i in raw_data.index:
            tokens = raw_data.text[i].split(' ')
            label_id = raw_data.hs[i]
            dataset.append((label_list + sep_token + tokens, label_id))
        self._dataset=dataset

    def __getitem__(self, index):
        return self._dataset[index]

    def __len__(self):
        return len(self._dataset)

def my_collate(batch, tokenizer, method, num_classes):
    tokens, label_ids = map(list, zip(*batch))
    text_ids = tokenizer(tokens,
                         padding=True,
                         truncation=True,
                         max_length=256,
                         is_split_into_words=True,
                         add_special_tokens=True,
                         return_tensors='pt')
    if method not in ['ce', 'scl']:
        positions = torch.zeros_like(text_ids['input_ids'])
        positions[:, num_classes:] = torch.arange(0, text_ids['input_ids'].size(1)-num_classes)
        text_ids['position_ids'] = positions
    return text_ids, torch.tensor(label_ids)

def load_data(dataset, data_dir, tokenizer, train_batch_size, test_batch_size, model_name, method, workers):
    if dataset == 'arabic':
        test_data = pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-mulki/L-HSAB.csv', sep = '\t')
        label_dict = {'normal': 0, 'hatful': 1}

    # load dataset .....................................................
    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-ousidhoum/ar_dataset.csv')
    pd_arabic['sentiment'][pd_arabic['sentiment']=='normal']=0
    pd_arabic['sentiment'][pd_arabic['sentiment']=='hateful']=1
    pd_arabic_n=pd_arabic[(pd_arabic['sentiment']==0) | (pd_arabic['sentiment']==1)]
    pd_arabic_n['text']=pd_arabic_n['tweet']
    pd_arabic_n['hs']=pd_arabic_n['sentiment']
    pd_arabic_n=pd_arabic_n[['text','hs']]
    pd_arabic_n.text =  pd_arabic_n.text.replace(np.NaN, regex=True)
    pd_arabic_n=pd_arabic_n.reset_index()


    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-mulki/L-HSAB.csv', sep = '\t')
    pd_arabic['Class'][pd_arabic['Class']=='normal']=0
    pd_arabic['Class'][pd_arabic['Class']=='hate']=1
    pd_arabic_m=pd_arabic[(pd_arabic['Class']==0) | (pd_arabic['Class']==1)]
    pd_arabic_m['text']=pd_arabic_m['Tweet']
    pd_arabic_m['hs']=pd_arabic_m['Class']
    pd_arabic_m=pd_arabic_m[['text','hs']]
    # X_test_sample=test_data.iloc[random.sample(range(0, len(test_data)), num_sample_test)];
    #pd_arabic_m.text =  pd_arabic_n.text.replace(np.NaN, regex=True)
    pd_arabic_m=pd_arabic_m.reset_index()

    #T2-HSAB-csv

    colnames=['tweet', 'label']
    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-haddad/M1_train_T_HSAB.csv')
    #pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-haddad/T3-HSAB.csv',names=colnames, header=None,sep=';')
    pd_arabic['label'][pd_arabic['label']=='normal']=0
    pd_arabic['label'][pd_arabic['label']=='hate']=1
    pd_arabic_t2=pd_arabic[(pd_arabic['label']==0) | (pd_arabic['label']==1)]
    pd_arabic_t2['text']=pd_arabic_t2['tweet']
    pd_arabic_t2['hs']=pd_arabic_t2['label']
    pd_arabic_t2=pd_arabic_t2[['text','hs']]
    pd_arabic_t2=pd_arabic_t2.reset_index()

    #T-HSAB-excel
    #colnames=['tweet', 'label']
    #pd_arabic= pd.read_excel(os.path.join(TrainData,TrainFiles[10]), names=colnames, header=None)
    #pd_arabic= pd.read_excel('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-haddad/T-HSAB.xlsx', names=colnames, header=None)
    #pd_arabic['label'][pd_arabic['label']=='normal']=0
    #pd_arabic['label'][pd_arabic['label']=='hate']=1
    #pd_arabic_t=pd_arabic[(pd_arabic['label']==0) | (pd_arabic['label']==1)]
    #pd_arabic_t['text']=pd_arabic_t['tweet']
    #pd_arabic_t['hs']=pd_arabic_t['label']
    #pd_arabic_t=pd_arabic_t[['text','hs']]
    #pd_arabic_t=pd_arabic_t.reset_index()

    arbic = pd.concat([pd_arabic_n,pd_arabic_m,pd_arabic_t2])
    from sklearn.model_selection import train_test_split
    Y_col = 'hs'
    X_cols = arbic.loc[:, arbic.columns != Y_col].columns
    X_train_arabic, X_test_arabic, y_train_arabic, y_test_arabic = train_test_split(arbic[X_cols], arbic[Y_col],test_size=0.20, random_state=42)
    X_train_arabic['hs']=y_train_arabic
    X_test_arabic['hs']=y_test_arabic

    #Urdo
    colnames=['tweet', 'label']
    #pd_urdo= pd.read_excel(os.path.join(TrainData,TrainFiles[10]), names=colnames, header=None)
    pd_urdo= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/urdo/task_2_train.tsv', names=colnames, header=None, sep = '\t')
    pd_urdo_t=pd_urdo[(pd_urdo['label']==1) | (pd_urdo['label']==2)| (pd_urdo['label']==3)]
    pd_urdo_t['text']=pd_urdo_t['tweet']
    pd_urdo_t['hs']=pd_urdo_t['label']
    pd_urdo_t=pd_urdo_t[['text','hs']]
    pd_urdo_t['hs'][pd_urdo_t['hs']==1]='normal'
    pd_urdo_t['hs'][pd_urdo_t['hs']==2] ='hateful'
    pd_urdo_t['hs'][pd_urdo_t['hs']==3] ='hateful'
    pd_urdo_t['hs'][pd_urdo_t['hs']=='normal']=0
    pd_urdo_t['hs'][pd_urdo_t['hs']=='hateful']=1
    pd_urdo_t=pd_urdo_t.reset_index()
    Y_col = 'hs'
    X_cols = pd_urdo_t.loc[:, pd_urdo_t.columns != Y_col].columns
    X_train_urdo, X_test_urdo, y_train_urdo, y_test_urdo = train_test_split(pd_urdo_t[X_cols], pd_urdo_t[Y_col],test_size=0.20, random_state=42)
    X_train_urdo['hs']=y_train_urdo
    X_test_urdo['hs']=y_test_urdo



    pd_persian= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/Persian_HS-main/persian_HS.csv')
    pd_persian['label'][pd_persian['label']==pd_persian.label[2]]=0
    pd_persian['label'][pd_persian['label']==pd_persian.label[5]]=1
    pd_persian_n=pd_persian[(pd_persian['label']==0) | (pd_persian['label']==1)]
    pd_persian_n['text']=pd_persian_n['sentence']
    pd_persian_n['hs']=pd_persian_n['label']
    pd_persian_n=pd_persian_n[['text','hs']]
    pd_persian_n.text =  pd_persian_n.text.replace(np.NaN, regex=True)
    pd_persian_n=pd_persian_n.reset_index()
    Y_col = 'hs'
    X_cols = pd_persian_n.loc[:, pd_persian_n.columns != Y_col].columns
    X_train_persian, X_test_persian, y_train_persian, y_test_persian = train_test_split(pd_persian_n[X_cols], pd_persian_n[Y_col],test_size=0.20, random_state=42)
    X_train_persian['hs']=y_train_persian
    X_test_persian['hs']=y_test_persian

    X_train_final = arbic
    X_test_final = X_test_persian
    label_dict = {'normal': 0, 'hatful': 1}
    X_train_final

    trainset = MyDataset(X_train_final.reset_index(),label_dict, tokenizer, model_name, method)
    testset = MyDataset(X_test_final.reset_index(), label_dict, tokenizer, model_name, method)
    collate_fn = partial(my_collate, tokenizer=tokenizer, method=method, num_classes=len(label_dict))
    train_dataloader = DataLoader(trainset, train_batch_size, shuffle=True, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    test_dataloader = DataLoader(testset, test_batch_size, shuffle=False, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    return train_dataloader, test_dataloader

In [ ]:
ins = Instructor(args, logger)
ins.run()

INFO:root:> creating model mbert


> creating model mbert
> creating model mbert


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:> cuda memory allocated: 711706112


> cuda memory allocated: 711706112
> cuda memory allocated: 711706112


INFO:root:> training arguments:


...

[test] loss: 3.9916, acc: 68.50
[test] loss: 3.9916, acc: 68.50


INFO:root:best loss: 3.1781, best acc: 68.93


best loss: 3.1781, best acc: 68.93
best loss: 3.1781, best acc: 68.93


INFO:root:log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log


log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log
log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log


# **few-shot**

In [ ]:
class MyDataset(Dataset):

    def __init__(self, raw_data, label_dict, tokenizer, model_name, method):
        label_list = ['positive', 'negative'] if method not in ['ce', 'scl'] else []
        sep_token = ['[SEP]'] if model_name == 'bert' else ['</s>']
        dataset = list()
        data=raw_data
        for i in raw_data.index:
            tokens = raw_data.text[i].split(' ')
            label_id = raw_data.hs[i]
            dataset.append((label_list + sep_token + tokens, label_id))
        self._dataset=dataset

    def __getitem__(self, index):
        return self._dataset[index]

    def __len__(self):
        return len(self._dataset)

def my_collate(batch, tokenizer, method, num_classes):
    tokens, label_ids = map(list, zip(*batch))
    text_ids = tokenizer(tokens,
                         padding=True,
                         truncation=True,
                         max_length=256,
                         is_split_into_words=True,
                         add_special_tokens=True,
                         return_tensors='pt')
    if method not in ['ce', 'scl']:
        positions = torch.zeros_like(text_ids['input_ids'])
        positions[:, num_classes:] = torch.arange(0, text_ids['input_ids'].size(1)-num_classes)
        text_ids['position_ids'] = positions
    return text_ids, torch.tensor(label_ids)

def load_data(dataset, data_dir, tokenizer, train_batch_size, test_batch_size, model_name, method, workers):
    if dataset == 'arabic':
        test_data = pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-mulki/L-HSAB.csv', sep = '\t')
        label_dict = {'normal': 0, 'hatful': 1}

    # load dataset .....................................................
    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-ousidhoum/ar_dataset.csv')
    pd_arabic['sentiment'][pd_arabic['sentiment']=='normal']=0
    pd_arabic['sentiment'][pd_arabic['sentiment']=='hateful']=1
    pd_arabic_n=pd_arabic[(pd_arabic['sentiment']==0) | (pd_arabic['sentiment']==1)]
    pd_arabic_n['text']=pd_arabic_n['tweet']
    pd_arabic_n['hs']=pd_arabic_n['sentiment']
    pd_arabic_n=pd_arabic_n[['text','hs']]
    pd_arabic_n.text =  pd_arabic_n.text.replace(np.NaN, regex=True)
    pd_arabic_n=pd_arabic_n.reset_index()


    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-mulki/L-HSAB.csv', sep = '\t')
    pd_arabic['Class'][pd_arabic['Class']=='normal']=0
    pd_arabic['Class'][pd_arabic['Class']=='hate']=1
    pd_arabic_m=pd_arabic[(pd_arabic['Class']==0) | (pd_arabic['Class']==1)]
    pd_arabic_m['text']=pd_arabic_m['Tweet']
    pd_arabic_m['hs']=pd_arabic_m['Class']
    pd_arabic_m=pd_arabic_m[['text','hs']]
    # X_test_sample=test_data.iloc[random.sample(range(0, len(test_data)), num_sample_test)];
    #pd_arabic_m.text =  pd_arabic_n.text.replace(np.NaN, regex=True)
    pd_arabic_m=pd_arabic_m.reset_index()

     #T2-HSAB-csv

    colnames=['tweet', 'label']
    #pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-haddad/M1_train_T_HSAB.csv')
    pd_arabic= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/arabic/arabic-haddad/T3-HSAB.csv',names=colnames, header=None,sep=';')
    pd_arabic['label'][pd_arabic['label']=='normal']=0
    pd_arabic['label'][pd_arabic['label']=='hate']=1
    pd_arabic_t2=pd_arabic[(pd_arabic['label']==0) | (pd_arabic['label']==1)]
    pd_arabic_t2['text']=pd_arabic_t2['tweet']
    pd_arabic_t2['hs']=pd_arabic_t2['label']
    pd_arabic_t2=pd_arabic_t2[['text','hs']]
    pd_arabic_t2=pd_arabic_t2.reset_index()

    arbic = pd.concat([pd_arabic_n,pd_arabic_m,pd_arabic_t2])
    from sklearn.model_selection import train_test_split
    Y_col = 'hs'
    X_cols = arbic.loc[:, arbic.columns != Y_col].columns
    X_train_arabic, X_test_arabic, y_train_arabic, y_test_arabic = train_test_split(arbic[X_cols], arbic[Y_col],test_size=0.20, random_state=42)
    X_train_arabic['hs']=y_train_arabic
    X_test_arabic['hs']=y_test_arabic


    pd_persian= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/Persian_HS-main/persian_HS.csv')
    pd_persian['label'][pd_persian['label']==pd_persian.label[2]]=0
    pd_persian['label'][pd_persian['label']==pd_persian.label[5]]=1
    pd_persian_n=pd_persian[(pd_persian['label']==0) | (pd_persian['label']==1)]
    pd_persian_n['text']=pd_persian_n['sentence']
    pd_persian_n['hs']=pd_persian_n['label']
    pd_persian_n=pd_persian_n[['text','hs']]
    pd_persian_n.text =  pd_persian_n.text.replace(np.NaN, regex=True)
    pd_persian_n=pd_persian_n.reset_index()
    Y_col = 'hs'
    X_cols = pd_persian_n.loc[:, pd_persian_n.columns != Y_col].columns
    X_train_persian, X_test_persian, y_train_persian, y_test_persian = train_test_split(pd_persian_n[X_cols], pd_persian_n[Y_col],test_size=0.95, random_state=42)
    X_train_persian['hs']=y_train_persian
    X_test_persian['hs']=y_test_persian
    X_train_persian_2, X_test_persian_2, y_train_persian_2, y_test_persian_2 = train_test_split(X_test_persian[X_cols], X_test_persian[Y_col],test_size=0.20, random_state=42)
    X_train_persian_2['hs']=y_train_persian_2
    X_test_persian_2['hs']=y_test_persian_2

     #Urdo
    colnames=['tweet', 'label']
    #pd_urdo= pd.read_excel(os.path.join(TrainData,TrainFiles[10]), names=colnames, header=None)
    pd_urdo= pd.read_csv('/content/gdrive/MyDrive/Hate_Speech/dataset/hate_dataset/urdo/task_2_train.tsv', names=colnames, header=None, sep = '\t')
    pd_urdo_t=pd_urdo[(pd_urdo['label']==1) | (pd_urdo['label']==2)| (pd_urdo['label']==3)]
    pd_urdo_t['text']=pd_urdo_t['tweet']
    pd_urdo_t['hs']=pd_urdo_t['label']
    pd_urdo_t=pd_urdo_t[['text','hs']]
    pd_urdo_t['hs'][pd_urdo_t['hs']==1]='normal'
    pd_urdo_t['hs'][pd_urdo_t['hs']==2] ='hateful'
    pd_urdo_t['hs'][pd_urdo_t['hs']==3] ='hateful'
    pd_urdo_t['hs'][pd_urdo_t['hs']=='normal']=0
    pd_urdo_t['hs'][pd_urdo_t['hs']=='hateful']=1
    pd_urdo_t=pd_urdo_t.reset_index()
    Y_col = 'hs'
    X_cols = pd_urdo_t.loc[:, pd_urdo_t.columns != Y_col].columns
    X_train_urdo, X_test_urdo, y_train_urdo, y_test_urdo = train_test_split(pd_urdo_t[X_cols], pd_urdo_t[Y_col],test_size=0.95, random_state=42)
    X_train_urdo['hs']=y_train_urdo
    X_test_urdo['hs']=y_test_urdo
    X_train_urdo_2, X_test_urdo_2, y_train_urdo_2, y_test_urdo_2 = train_test_split(X_test_urdo[X_cols], X_test_urdo[Y_col],test_size=0.20, random_state=42)
    X_train_urdo_2['hs']=y_train_urdo_2
    X_test_urdo_2['hs']=y_test_urdo_2

    frames_train = [arbic,X_train_persian]
    X_train_final = pd.concat(frames_train)
    #X_train_final = X_train_arabic
    X_test_final = X_test_persian_2
    label_dict = {'normal': 0, 'hatful': 1}
    X_train_final

    trainset = MyDataset(X_train_final.reset_index(),label_dict, tokenizer, model_name, method)
    testset = MyDataset(X_test_final.reset_index(), label_dict, tokenizer, model_name, method)
    collate_fn = partial(my_collate, tokenizer=tokenizer, method=method, num_classes=len(label_dict))
    train_dataloader = DataLoader(trainset, train_batch_size, shuffle=True, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    test_dataloader = DataLoader(testset, test_batch_size, shuffle=False, num_workers=workers, collate_fn=collate_fn, pin_memory=True)
    return train_dataloader, test_dataloader

In [ ]:
ins = Instructor(args, logger)
ins.run()

INFO:root:> creating model mbert


> creating model mbert
> creating model mbert


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:> cuda memory allocated: 2193759232


> cuda memory allocated: 2193759232
> cuda memory allocated: 2193759232


INFO:root:> training arguments:


...

[test] loss: 4.1053, acc: 70.93
[test] loss: 4.1053, acc: 70.93


INFO:root:best loss: 3.6228, best acc: 73.82


best loss: 3.6228, best acc: 73.82
best loss: 3.6228, best acc: 73.82


INFO:root:log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log


log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log
log saved: arabic_mbert_Adversarial_23-06-26_10-31-30.log
